In [21]:
pip install spotipy pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tqdm


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
from tqdm import tqdm  # for progress bars

# ----------------------------
# CONFIGURATION
# ----------------------------
CLIENT_ID = "0c136fb58a17412f94c81a35e109360d"
CLIENT_SECRET = "0289c7b9332c43ef80fd34b806d9ce76"

OUTPUT_FILE = "spotify_songs.csv"

# Mix of different playlist formats - all should work now
playlists = [
    "37i9dQZF1DWXRqgorJj26U",  # Rock Classics (if still available)
    "5iwkYfnHAGMEFLiHFFGnP4",  # HITS 2025 - Today's Top Songs
    "0MjxRIyt08vQ0QPDE02Zj2",  # Hits 2025
    "3Me7esQS0xZkSbW0XW7roB",  # 2025 HITS 🔥🔥🔥
    "1YUVG3tWJL8bYRpNiRFd29"
    # You can add more playlists here
]

# ----------------------------
# HELPER FUNCTION
# ----------------------------
def extract_playlist_id(playlist_input: str) -> str:
    """Extract playlist ID from URL, URI, or ID format."""
    if playlist_input.startswith("spotify:playlist:"):
        return playlist_input.split(":")[-1]
    if "open.spotify.com/playlist/" in playlist_input:
        return playlist_input.split("playlist/")[-1].split("?")[0]
    return playlist_input  # Already an ID

def get_playlist_info(sp, playlist_id):
    """Get basic playlist information."""
    try:
        playlist = sp.playlist(playlist_id, fields="name,owner.display_name,public,tracks.total")
        return {
            'name': playlist['name'],
            'owner': playlist['owner']['display_name'],
            'public': playlist['public'],
            'total_tracks': playlist['tracks']['total']
        }
    except:
        return None

# ----------------------------
# AUTHENTICATION
# ----------------------------
print("🔐 Authenticating with Spotify...")
try:
    # Using ClientCredentials for public data access (no user login required)
    client_credentials_manager = SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    print("✅ Successfully authenticated with Spotify!")
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    exit(1)

# ----------------------------
# REMOVE OLD CSV (if exists)
# ----------------------------
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)
    print(f"🧹 Removed old database file '{OUTPUT_FILE}'")

# ----------------------------
# FETCH SONGS
# ----------------------------
all_tracks = []
successful_playlists = 0

print("\n🚀 Starting to fetch song data from Spotify playlists...")

for i, playlist in enumerate(playlists, 1):
    try:
        playlist_id = extract_playlist_id(playlist)
        print(f"\n[{i}/{len(playlists)}] Processing playlist: {playlist_id}")
        
        # Get playlist info first
        playlist_info = get_playlist_info(sp, playlist_id)
        if not playlist_info:
            print(f"  ❌ Could not access playlist {playlist_id}")
            continue
            
        if not playlist_info['public']:
            print(f"  ⚠️  Playlist '{playlist_info['name']}' is private, skipping...")
            continue
            
        print(f"  📋 Playlist: '{playlist_info['name']}' by {playlist_info['owner']}")
        print(f"  📊 Total tracks: {playlist_info['total_tracks']}")

        # Fetch tracks with pagination
        results = sp.playlist_tracks(playlist_id, limit=100, offset=0)
        tracks = results["items"]

        # Progress bar
        pbar = tqdm(
            total=playlist_info['total_tracks'], 
            desc=f"  Fetching tracks", 
            unit="track",
            leave=False
        )
        pbar.update(len(tracks))

        # Handle pagination
        page = 1
        max_pages = 200  # Safety limit
        while results["next"] and page < max_pages:
            try:
                results = sp.next(results)
                page += 1
                new_tracks = results["items"]
                tracks.extend(new_tracks)
                pbar.update(len(new_tracks))
            except Exception as e:
                print(f"  ⚠️  Error fetching page {page}: {e}")
                break

        pbar.close()

        # Process tracks
        playlist_tracks = []
        for item in tracks:
            track = item.get("track")
            if track and track.get("id"):  # Ensure track exists and has ID
                try:
                    playlist_tracks.append({
                        "playlist_id": playlist_id,
                        "playlist_name": playlist_info['name'],
                        "track_id": track["id"],
                        "track_name": track["name"],
                        "artist": ", ".join([artist["name"] for artist in track["artists"]]),
                        "album": track["album"]["name"],
                        "release_date": track["album"]["release_date"],
                        "popularity": track.get("popularity", 0),
                        "duration_ms": track.get("duration_ms", 0),
                        "explicit": track.get("explicit", False),
                        "track_url": track["external_urls"]["spotify"]
                    })
                except Exception as e:
                    print(f"  ⚠️  Error processing track: {e}")
                    continue

        all_tracks.extend(playlist_tracks)
        successful_playlists += 1
        print(f"  ✅ Successfully fetched {len(playlist_tracks)} tracks")
        print(f"  📈 Running total: {len(all_tracks)} tracks")

    except Exception as e:
        print(f"  ❌ Could not process playlist {playlist}. Error: {e}")

print(f"\n📊 Summary:")
print(f"  • Processed {successful_playlists}/{len(playlists)} playlists successfully")
print(f"  • Total tracks fetched: {len(all_tracks)}")

# ----------------------------
# SAVE TO CSV
# ----------------------------
if all_tracks:
    # Create DataFrame and remove duplicates
    df = pd.DataFrame(all_tracks)
    original_count = len(df)
    
    # Remove duplicate tracks (same track_id)
    df = df.drop_duplicates(subset=['track_id'], keep='first')
    unique_count = len(df)
    duplicates_removed = original_count - unique_count
    
    # Save to CSV
    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
    
    print(f"\n✅ Success!")
    print(f"  • Removed {duplicates_removed} duplicate tracks")
    print(f"  • Saved {unique_count} unique tracks to '{OUTPUT_FILE}'")
    print(f"\n📋 Dataset preview:")
    print(df.head())
    print(f"\n📈 Dataset info:")
    print(f"  • Columns: {list(df.columns)}")
    print(f"  • Artists: {df['artist'].nunique()} unique")
    print(f"  • Albums: {df['album'].nunique()} unique")
else:
    print("\n❌ No tracks were fetched.")
    print("💡 Troubleshooting tips:")
    print("  1. Check if your CLIENT_ID and CLIENT_SECRET are valid")
    print("  2. Try with different (public) playlist IDs")
    print("  3. Some playlists might be region-restricted")

🔐 Authenticating with Spotify...
✅ Successfully authenticated with Spotify!

🚀 Starting to fetch song data from Spotify playlists...

[1/5] Processing playlist: 37i9dQZF1DWXRqgorJj26U


HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWXRqgorJj26U with Params: {'fields': 'name,owner.display_name,public,tracks.total', 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


  ❌ Could not access playlist 37i9dQZF1DWXRqgorJj26U

[2/5] Processing playlist: 5iwkYfnHAGMEFLiHFFGnP4
  📋 Playlist: 'HITS 2025 - Today's Top Songs' by Topsify
  📊 Total tracks: 60


  ✅ Successfully fetched 60 tracks
  📈 Running total: 60 tracks

[3/5] Processing playlist: 0MjxRIyt08vQ0QPDE02Zj2
  📋 Playlist: 'Hits 2025' by my.charts.music
  📊 Total tracks: 807


  ✅ Successfully fetched 807 tracks
  📈 Running total: 867 tracks

[4/5] Processing playlist: 3Me7esQS0xZkSbW0XW7roB
  📋 Playlist: '2025 HITS 🔥 🔥🔥 BEST HITS 2025 / Top Hits ' by partyfiesta!
  📊 Total tracks: 247


  ✅ Successfully fetched 247 tracks
  📈 Running total: 1114 tracks

[5/5] Processing playlist: 1YUVG3tWJL8bYRpNiRFd29
  📋 Playlist: 'Telugu Playlist' by karthik varma
  📊 Total tracks: 61


  ✅ Successfully fetched 61 tracks
  📈 Running total: 1175 tracks

📊 Summary:
  • Processed 4/5 playlists successfully
  • Total tracks fetched: 1175

✅ Success!
  • Removed 97 duplicate tracks
  • Saved 1078 unique tracks to 'spotify_songs.csv'

📋 Dataset preview:
              playlist_id                  playlist_name  \
0  5iwkYfnHAGMEFLiHFFGnP4  HITS 2025 - Today's Top Songs   
1  5iwkYfnHAGMEFLiHFFGnP4  HITS 2025 - Today's Top Songs   
2  5iwkYfnHAGMEFLiHFFGnP4  HITS 2025 - Today's Top Songs   
3  5iwkYfnHAGMEFLiHFFGnP4  HITS 2025 - Today's Top Songs   
4  5iwkYfnHAGMEFLiHFFGnP4  HITS 2025 - Today's Top Songs   

                 track_id                               track_name  \
0  6qqrTXSdwiJaq8SO0X2lSe                                 Ordinary   
1  4WFgvKVfEhb3IUAFGrutTR                              Love Me Not   
2  05od2qm2MTSKCHxy1GBp5W                                 12 to 12   
3  1CPZ5BxNNd0n0nF4Orb9JS                                   Golden   
4  2yWlGEgEfPot0lv3OAju

🔐 Authenticating...
✅ Authenticated as: Shashank

🚀 Starting to fetch song data...

-> Processing playlist: 5iwkYfnHAGMEFLiHFFGnP4


   Fetching audio features:   0%|          | 0/1 [00:00<?, ?it/s]


   ❌ Could not process playlist 5iwkYfnHAGMEFLiHFFGnP4. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6qqrTXSdwiJaq8SO0X2lSe,4WFgvKVfEhb3IUAFGrutTR,05od2qm2MTSKCHxy1GBp5W,1CPZ5BxNNd0n0nF4Orb9JS,2yWlGEgEfPot0lv3OAjuG3,7uivYk4Bf2j1wtHHLtB0Qr,3cZajhyr8LmtPfHZ9296tj,42VUCXerQ5qTr4Qp6PhKo4,5q9I5RmmrLC4U2mW2BnF3K,7qjZnBKE73H4Oxkopwulqe,2SsY5k7UWFqgye3PUMG3Oq,221eHga9f9Ne4f8D7WVgPT,5BZsQlgw21vDOAjoqkNgKb,4wJ5Qq0jBN4ajy7ouZIV1c,6wHpLMmp98aHcV8L1JFrj8,04emojnbYkrRmv5qtJcgVP,6vqyk3mbDBv3npTpctYoka,1IfGOnXoMBkiKC9xljlSfa,1WmBVbFmLt0w6zPP37TeCG,5H1sKFMzDeMtXwND3V6hRY,3nHuKdFJZm78CoeBnDcFKe,7so0lgd0zP2Sbgs2d7a1SZ,0InIeZW4P6VO7dUGRM4AKH,5ITV0zqzjOYfFWpW0xBmRa,6iOndD4OFo7GkaDypWQIou,2qPQcfUgizNDOIv1orkTAd,2CGNAOSuO1MEFCbBRgUzjd,4TwEdnSiTPDR1vg1QZ5K8W,6dOtVTDdiauQNBQEDOtlAB,4Q0qVhFQa7j6jRKzo3HDmP,0Ji9UonfwC90rbZ4IaQhOb,0TFTAtCYhp2tQ9KcJIZb55,5pobX21I8uZdHM8YOWItU7,3xkHsmpQCBMytMJNiDf3Ii,42UBPzRMh5yyz0EDPr6fr1,1zM8S2pnIclcdTmrfFnWtX,1qbmS6ep2hbBRaEZFpn7BX,0bxPRWprUV

   Fetching audio features:   0%|          | 0/9 [00:00<?, ?it/s]


   ❌ Could not process playlist 0MjxRIyt08vQ0QPDE02Zj2. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6qqrTXSdwiJaq8SO0X2lSe,1CPZ5BxNNd0n0nF4Orb9JS,35ISBknsCeZQtq66xABI9g,0GRc3eGTg8HBdWLRGYgqIc,6j8OrKDQHxXakjA7KKbWzj,0FTmksd2dxiE5e3rWyJXs6,4oASO8jyg0Qh7cUeve1bEA,6wHpLMmp98aHcV8L1JFrj8,2yWlGEgEfPot0lv3OAjuG3,5vNRhkKd0yEAg8suGBpjeY,1musbempyJAw5gfSKZHXP9,5ITV0zqzjOYfFWpW0xBmRa,6rcuy6XqlB9bqvtIRDl25u,4Q0qVhFQa7j6jRKzo3HDmP,2GN2pbQi4Yz7Qp99Wbp2vZ,5SxahezRlC0saXbCALfB7c,4AajxCEwGEsmHmT4H1TwjY,1UNEuG9DYOWiikf00ayr52,2262bWmqomIaJXwCRHr13j,1We3jgIztlD2Tkb0yOoX4b,2sKHevALE8DKDkwEo04Pbh,5ejTcCJOozPqRPMs9gW3pN,5nPbKG04fhLkIAjcPFaZq7,6Ilomcr9iLplsgAvZjHkJP,78nx0HDJIFD5xDq2L5420Z,0QCIpQV3twfqo9kh0t8Zza,42UBPzRMh5yyz0EDPr6fr1,7ETIZFr7163zFcz3RxKk5O,3Qt9dFYxJLA4u2rlXafqrf,7zPfFxH8WxUUffU8rqB4kU,1xOqGUkyxGQRdCvGpvWKmL,1dPfUpXRn8U5Q1fgAnhvbe,6V7bxWg4mmuU4ILbYPQJvd,1GfJCWGLfdev016PSsrUO8,2MZeoO5my5Z2opiQHm150R,1WmBVbFmLt0w6zPP37TeCG,7ne4VBA60CxGM75vw0EYad,5ZLUm9eab8

   Fetching audio features:   0%|          | 0/3 [00:00<?, ?it/s]

   ❌ Could not process playlist 3Me7esQS0xZkSbW0XW7roB. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2RkZ5LkEzeHGRsmDqKwmaJ,4WFgvKVfEhb3IUAFGrutTR,0FTmksd2dxiE5e3rWyJXs6,0je57Uq5eTk1wrPzn9sWbl,2plbrEY59IikOBgBGLjaoe,5BZsQlgw21vDOAjoqkNgKb,1WmBVbFmLt0w6zPP37TeCG,4Q0qVhFQa7j6jRKzo3HDmP,5vNRhkKd0yEAg8suGBpjeY,27xkOIER6uDLKALIelHylZ,3cZajhyr8LmtPfHZ9296tj,2SsY5k7UWFqgye3PUMG3Oq,0ILLfGp9z1pukpjifsfkxx,42VUCXerQ5qTr4Qp6PhKo4,2yWlGEgEfPot0lv3OAjuG3,1mEQbSq0PCNjyL6oMgHKFQ,2RdEC8Ff83WkX7kDVCHseE,56sExwBBmnDoRErXqghqrW,42UBPzRMh5yyz0EDPr6fr1,5ITV0zqzjOYfFWpW0xBmRa,1musbempyJAw5gfSKZHXP9,56n4ZSuQrwdLjfx4UUrMvP,05od2qm2MTSKCHxy1GBp5W,1CPZ5BxNNd0n0nF4Orb9JS,6tNQ70jh4OwmPGpYy6R2o9,50s7qiLWfzgpAsDqwfjS5D,3AWDeHLc88XogCaCnZQLVI,5ZLUm9eab8y3tqQ1OhQSHI,2CGNAOSuO1MEFCbBRgUzjd,7wMnlYWFlogro7Xv2pgmUj,7ne4VBA60CxGM75vw0EYad,4AajxCEwGEsmHmT4H1TwjY,6dOtVTDdiauQNBQEDOtlAB,2vMXaguuN4UKXfoRlxySuG,0fK7ie6XwGxQTIkpFoWkd1,0GRc3eGTg8HBdWLRGYgqIc,1qbmS6ep2hbBRaEZFpn7BX,5P9nLjjpKc

In [18]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from tqdm import tqdm

# -----------------------
# CONFIG
# -----------------------
CLIENT_ID = "0c136fb58a17412f94c81a35e109360d"
CLIENT_SECRET = "0289c7b9332c43ef80fd34b806d9ce76"
REDIRECT_URI = "https://127.0.0.1:8888/callback"
SCOPE = ""

PLAYLISTS = [
    "37i9dQZF1DX4JAvHpjipBk",  # Rock Classics
    "37i9dQZF1DXcF6B6QPhFDv"   # Hot Hits India
]

OUTPUT_FILE = "spotify_tracks_with_features.csv"

# -----------------------
# AUTHENTICATION
# -----------------------
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

# -----------------------
# HELPER FUNCTION
# -----------------------
def get_playlist_tracks(playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id, limit=100)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# -----------------------
# MAIN LOOP
# -----------------------
all_tracks = []

for playlist_id in PLAYLISTS:
    print(f"\n🚀 Processing playlist: {playlist_id}")
    try:
        tracks = get_playlist_tracks(playlist_id)
        track_ids = [t['track']['id'] for t in tracks if t['track'] and t['track']['id']]

        print(f"🎵 Found {len(track_ids)} tracks. Fetching audio features...")

        # Spotify API allows 100 tracks per request
        for i in tqdm(range(0, len(track_ids), 100)):
            batch_ids = track_ids[i:i+100]
            features = sp.audio_features(batch_ids)
            for f in features:
                if f:  # skip None
                    track_info = next((t['track'] for t in tracks if t['track']['id'] == f['id']), {})
                    all_tracks.append({
                        "track_name": track_info.get('name'),
                        "artist": ", ".join([a['name'] for a in track_info.get('artists', [])]),
                        "album": track_info.get('album', {}).get('name'),
                        "release_date": track_info.get('album', {}).get('release_date'),
                        "track_id": f['id'],
                        "valence": f['valence'],
                        "energy": f['energy'],
                        "danceability": f['danceability'],
                        "tempo": f['tempo'],
                        "track_url": track_info.get('external_urls', {}).get('spotify')
                    })

    except Exception as e:
        print(f"❌ Failed to fetch playlist {playlist_id}: {e}")

# -----------------------
# SAVE CSV
# -----------------------
if all_tracks:
    df = pd.DataFrame(all_tracks)
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n✅ Saved {len(all_tracks)} tracks to '{OUTPUT_FILE}'")
else:
    print("\n❌ No tracks were fetched.")



🚀 Processing playlist: 37i9dQZF1DX4JAvHpjipBk


HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DX4JAvHpjipBk/tracks with Params: {'limit': 100, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


❌ Failed to fetch playlist 37i9dQZF1DX4JAvHpjipBk: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZF1DX4JAvHpjipBk/tracks?limit=100&offset=0&additional_types=track:
 Resource not found, reason: None

🚀 Processing playlist: 37i9dQZF1DXcF6B6QPhFDv


HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DXcF6B6QPhFDv/tracks with Params: {'limit': 100, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


❌ Failed to fetch playlist 37i9dQZF1DXcF6B6QPhFDv: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZF1DXcF6B6QPhFDv/tracks?limit=100&offset=0&additional_types=track:
 Resource not found, reason: None

❌ No tracks were fetched.


In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm

# --- 1. SETUP & AUTHENTICATION ---
# This uses the simple server-to-server auth, which should work now.
load_dotenv("../backend/.env")
CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
INPUT_FILE = "spotify_songs.csv"
OUTPUT_FILE = "spotify_songs_complete.csv"

print("🔐 Authenticating...")
try:
    auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    print("✅ Authentication successful!")
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    exit()

# --- 2. LOAD YOUR EXISTING DATA ---
print(f"\n🎵 Loading tracks from '{INPUT_FILE}'...")
df_tracks = pd.read_csv(INPUT_FILE)
track_ids = df_tracks['track_id'].dropna().unique().tolist()
print(f"   Found {len(track_ids)} unique tracks to process.")

# --- 3. FETCH AND MERGE AUDIO FEATURES ---
audio_features_list = []
print("\n🚀 Fetching audio features in batches...")
for i in tqdm(range(0, len(track_ids), 100)):
    batch = track_ids[i:i+100]
    try:
        features = sp.audio_features(batch)
        audio_features_list.extend([f for f in features if f])
    except Exception as e:
        print(f"   ⚠️ Could not fetch batch {i//100 + 1}. Error: {e}")

if audio_features_list:
    df_features = pd.DataFrame(audio_features_list)
    df_features.rename(columns={'id': 'track_id'}, inplace=True)

    # Merge the original data with the new audio features
    df_complete = pd.merge(df_tracks, df_features, on='track_id', how='inner')

    # --- 4. SAVE THE FINAL, COMPLETE DATASET ---
    df_complete.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
    print(f"\n🎉 SUCCESS! Saved {len(df_complete)} tracks with audio features to '{OUTPUT_FILE}'.")
    print("\n📋 Preview of the final dataset:")
    print(df_complete[['track_name', 'artist', 'valence', 'energy', 'danceability']].head())
else:
    print("\n❌ FAILED: No audio features were fetched. Please check your connection or credentials.")

🔐 Authenticating...
✅ Authentication successful!

🎵 Loading tracks from 'spotify_songs.csv'...
   Found 1078 unique tracks to process.

🚀 Fetching audio features in batches...


 18%|█▊        | 2/11 [00:00<00:01,  4.61it/s]

   ⚠️ Could not fetch batch 1. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6qqrTXSdwiJaq8SO0X2lSe,4WFgvKVfEhb3IUAFGrutTR,05od2qm2MTSKCHxy1GBp5W,1CPZ5BxNNd0n0nF4Orb9JS,2yWlGEgEfPot0lv3OAjuG3,7uivYk4Bf2j1wtHHLtB0Qr,3cZajhyr8LmtPfHZ9296tj,42VUCXerQ5qTr4Qp6PhKo4,5q9I5RmmrLC4U2mW2BnF3K,7qjZnBKE73H4Oxkopwulqe,2SsY5k7UWFqgye3PUMG3Oq,221eHga9f9Ne4f8D7WVgPT,5BZsQlgw21vDOAjoqkNgKb,4wJ5Qq0jBN4ajy7ouZIV1c,6wHpLMmp98aHcV8L1JFrj8,04emojnbYkrRmv5qtJcgVP,6vqyk3mbDBv3npTpctYoka,1IfGOnXoMBkiKC9xljlSfa,1WmBVbFmLt0w6zPP37TeCG,5H1sKFMzDeMtXwND3V6hRY,3nHuKdFJZm78CoeBnDcFKe,7so0lgd0zP2Sbgs2d7a1SZ,0InIeZW4P6VO7dUGRM4AKH,5ITV0zqzjOYfFWpW0xBmRa,6iOndD4OFo7GkaDypWQIou,2qPQcfUgizNDOIv1orkTAd,2CGNAOSuO1MEFCbBRgUzjd,4TwEdnSiTPDR1vg1QZ5K8W,6dOtVTDdiauQNBQEDOtlAB,4Q0qVhFQa7j6jRKzo3HDmP,0Ji9UonfwC90rbZ4IaQhOb,0TFTAtCYhp2tQ9KcJIZb55,5pobX21I8uZdHM8YOWItU7,3xkHsmpQCBMytMJNiDf3Ii,42UBPzRMh5yyz0EDPr6fr1,1zM8S2pnIclcdTmrfFnWtX,1qbmS6ep2hbBRaEZFpn7BX,0bxPRWprUVpQK0UFcddkrA,57GsLpRtEtrz

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=75n9WHWZAzhB59xSjIHly4,2LwsunYgfRoqyIsNtgOCQx,3be9ACTxtcL6Zm4vJRUiPG,23H67W1NWkB4DOVVT8QOfV,5VfPYI5rj8GtQ2qChKYwMA,4lnLVhGxv1YnJEJIComb9x,4qR6ZdTuHGeXdWcKJB7sUA,7cMhicmOIwfJlWURl90AOc,5JLbgRkto4OrrkWgAHcJZT,5G2f63n7IPVPPjfNIGih7Q,726SEjbF3ToEbYbn43gnxl,69GEK9C8nKwiGLMv2NbpiG,2VyvDGdcVY04cNYou9MFVX,1T2zLWVcr4nXMrMnzyHUVl,3TPKsQTu9jZyzQJiax5rLA,39AfsBXaRa8JZhl46bKGGN,6jbYpRPTEFl1HFKHk1IC0m,3aSWXU6owkZeVhh94XxEWO,64f3yNXsi2Vk76odcHCOnw,3JG1uFc40wfyrqaWC7iv0e,4r6c2ClnuYLwNv4lEM7XlW,3ovTEUMYEoiT8ikEMVXBmS,7tI8dRuH2Yc6RuoTjxo4dU,7qQE1QwOsWHtTP08hw0y7u,6WatFBLVB0x077xWeoVc2k,51rfRCiUSvxXlCSCfIztBy,5is9ECympHEpAyTw22IDC5,39glDGcpBhLVMSsQhvtqHR,47KhhDku9RLDsWIJjYaXDs,20Y4EAmSXru3foatDg4OqN,0ngL9248WWfByTMRrZwUyz,19RybK6XDbAVpcdxSbZL1o,1bJRd2ey6ER7neSxZwYdAG,3WOhcATHxK2SLNeP5W3v1v,5496lks6V3fh1mZAB6IAap,2uqYupMHANxnwgeiXTZXzd,7MX0wxRiKE9c7dmiLV3NrX,0WbMK4wrZ1wFSty9F7FCgu,74KNUsROKQpivX8RHHwwr9,74lOK0Z3WKvTpK8W8Srj1d,3MZSuu8bD15

   ⚠️ Could not fetch batch 3. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=75n9WHWZAzhB59xSjIHly4,2LwsunYgfRoqyIsNtgOCQx,3be9ACTxtcL6Zm4vJRUiPG,23H67W1NWkB4DOVVT8QOfV,5VfPYI5rj8GtQ2qChKYwMA,4lnLVhGxv1YnJEJIComb9x,4qR6ZdTuHGeXdWcKJB7sUA,7cMhicmOIwfJlWURl90AOc,5JLbgRkto4OrrkWgAHcJZT,5G2f63n7IPVPPjfNIGih7Q,726SEjbF3ToEbYbn43gnxl,69GEK9C8nKwiGLMv2NbpiG,2VyvDGdcVY04cNYou9MFVX,1T2zLWVcr4nXMrMnzyHUVl,3TPKsQTu9jZyzQJiax5rLA,39AfsBXaRa8JZhl46bKGGN,6jbYpRPTEFl1HFKHk1IC0m,3aSWXU6owkZeVhh94XxEWO,64f3yNXsi2Vk76odcHCOnw,3JG1uFc40wfyrqaWC7iv0e,4r6c2ClnuYLwNv4lEM7XlW,3ovTEUMYEoiT8ikEMVXBmS,7tI8dRuH2Yc6RuoTjxo4dU,7qQE1QwOsWHtTP08hw0y7u,6WatFBLVB0x077xWeoVc2k,51rfRCiUSvxXlCSCfIztBy,5is9ECympHEpAyTw22IDC5,39glDGcpBhLVMSsQhvtqHR,47KhhDku9RLDsWIJjYaXDs,20Y4EAmSXru3foatDg4OqN,0ngL9248WWfByTMRrZwUyz,19RybK6XDbAVpcdxSbZL1o,1bJRd2ey6ER7neSxZwYdAG,3WOhcATHxK2SLNeP5W3v1v,5496lks6V3fh1mZAB6IAap,2uqYupMHANxnwgeiXTZXzd,7MX0wxRiKE9c7dmiLV3NrX,0WbMK4wrZ1wFSty9F7FCgu,74KNUsROKQpi

...

   ⚠️ Could not fetch batch 7. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=0NRIWkmQiSoIaEs3wvoSS7,1LPwPrqylnbchpPYUZsuZ8,0mhTXfQbfTOd8VGvnsTr1P,0WaszHB3WX7MrWhTDKJ8w0,3zAQQXsqLkD7ji4pQzKD0v,6XMNNNDOnkTXSh567RaGsb,0npEFzbFBxYea9nZIYUnmQ,03HuVUgzwdz4HNn6qiEPmJ,72PVbRKkUimweUPV0nKAPs,0smB5zD5CJYMtdXvS2qS5A,3eQsZyBNQSOAE5bzLH0JFt,0LutvoC2HqCfnaRSjrhtzo,39QvuWaRVyyLT9yQKn8Qy0,16XLWurcZVM96mYBgtPz4s,3pgwQNwEqbU12kLNXNDB7Q,4y6rZU5JCcDrksYDq7CMAY,4GojOaaXHYF8sX4u4jLjYJ,0LPqVcXdVWsiGz7SkyOrC4,0Upay8wgPd30hQPjeWXHOC,45Exz93oyRbDvmM8FvpgYz,0SMFOOsqD1jicOhuorY1vz,4uVLSm8PxZI3R76vbbeppn,5emOdyBZqKrL06yqZsQKFZ,2dworjMwgs5pglkFzszCUe,6U5AgSYUubKMrMDEazfSsF,13AzsOOlv5lwEeKsI1pAUt,4bylJIKOGyjXrQ6voXILh9,3awCCyQfwNbQdf3lL5EL7C,0kvklDSNAkcENUPfEvQNvC,4ZvwRwdFa07GQy64zSy8tY,4sMR8fuTzbUyzB0asxifV0,0eJSm2G0t3XX4SUuU1aSaH,7iwPGVfoiedHFNHvpxTznL,5ve67AkrXN3B6Gx7tHfz76,6MrLkXsMmHaYt680fhJUAq,3hH3ypxqE0JwYq4mfyYmlD,2sL7gSkWNsHU3zEMT67edU,6eDKyG6jvCXaTyTdFOh9wA,2Di98CfbonuW

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=44U1bVG6imxYfDUtd8KvTZ,4uQnwt6cBokQsIRKIl1VOT,38bCx6AbjjduaSVlgIwKZP,0ItVps4GAVBmOg8o9ZHEip,4DQNdeDEnbJIckoZ8XLg7G,3E3UHWv6I1WD2PqBjbeXn7,3ZZyUf7WkhfN1JkQZZ00fI,47mNWUCgJoXqVWb3UZytVa,0LgdXFDT9AfjMyaeoxBT2P,2GAZrSRYZQKyJM0VOBpS3U,4yFPKNzjkCS0PpRzhcvy1d,4XKCwXacGJ0Am1s7oD2rDM,4s1jXpACCXFDNYTKWKH3tq,41goAg1ZxePWjXXfZ7PhG7,4fEdyp5iZ7I8sWI9jcBSbp,2kpUNuingcchFIhLrbWxIO,3SOaoZ1AfhMjEMCJrdGKEw,7nHIOaMV7RMXkViqwl7lLT,5pe8674S8FnKW8tNpHFjAZ,290eKCMBc9f9CcSLFuQrim,2ETQsbiuGm2WLOSOr2I19A,5ehXToeJ8Tgc4wMhY42Oul,717s9KFmHhyy5t3xkStS56,28odUt3kXBD3wObHf87GVv,5DcEQDn2P3fmTHEOMSvbhi,42GKyvz5KBsHTBaLpo3cqJ,6islh8bFrZUUuFFCveWPfl,3vbFjvzmJLrdttGuDhGrzi,4b3W0IbHDwHCvjSs708Tr4,2RkZ5LkEzeHGRsmDqKwmaJ,0je57Uq5eTk1wrPzn9sWbl,0ILLfGp9z1pukpjifsfkxx,56sExwBBmnDoRErXqghqrW,56n4ZSuQrwdLjfx4UUrMvP,6tNQ70jh4OwmPGpYy6R2o9,50s7qiLWfzgpAsDqwfjS5D,7wMnlYWFlogro7Xv2pgmUj,2vMXaguuN4UKXfoRlxySuG,5P9nLjjpKczdnhypJa5teO,35J5md68GWAKbQ1Ys94S10,6393yGahSJ9

   ⚠️ Could not fetch batch 9. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=44U1bVG6imxYfDUtd8KvTZ,4uQnwt6cBokQsIRKIl1VOT,38bCx6AbjjduaSVlgIwKZP,0ItVps4GAVBmOg8o9ZHEip,4DQNdeDEnbJIckoZ8XLg7G,3E3UHWv6I1WD2PqBjbeXn7,3ZZyUf7WkhfN1JkQZZ00fI,47mNWUCgJoXqVWb3UZytVa,0LgdXFDT9AfjMyaeoxBT2P,2GAZrSRYZQKyJM0VOBpS3U,4yFPKNzjkCS0PpRzhcvy1d,4XKCwXacGJ0Am1s7oD2rDM,4s1jXpACCXFDNYTKWKH3tq,41goAg1ZxePWjXXfZ7PhG7,4fEdyp5iZ7I8sWI9jcBSbp,2kpUNuingcchFIhLrbWxIO,3SOaoZ1AfhMjEMCJrdGKEw,7nHIOaMV7RMXkViqwl7lLT,5pe8674S8FnKW8tNpHFjAZ,290eKCMBc9f9CcSLFuQrim,2ETQsbiuGm2WLOSOr2I19A,5ehXToeJ8Tgc4wMhY42Oul,717s9KFmHhyy5t3xkStS56,28odUt3kXBD3wObHf87GVv,5DcEQDn2P3fmTHEOMSvbhi,42GKyvz5KBsHTBaLpo3cqJ,6islh8bFrZUUuFFCveWPfl,3vbFjvzmJLrdttGuDhGrzi,4b3W0IbHDwHCvjSs708Tr4,2RkZ5LkEzeHGRsmDqKwmaJ,0je57Uq5eTk1wrPzn9sWbl,0ILLfGp9z1pukpjifsfkxx,56sExwBBmnDoRErXqghqrW,56n4ZSuQrwdLjfx4UUrMvP,6tNQ70jh4OwmPGpYy6R2o9,50s7qiLWfzgpAsDqwfjS5D,7wMnlYWFlogro7Xv2pgmUj,2vMXaguuN4UKXfoRlxySuG,5P9nLjjpKczd

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=39JofJHEtg8I4fSyo7Imft,6JVOeKaRMwFjgiH3IbZhW1,0JXXNGljqupsJaZsgSbMZV,1SRw5p2lVAi7RGlHEmZg66,6dgUya35uo964z7GZXM07g,1blLiE48uAhJAebV9dnODK,1qEmFfgcLObUfQm0j1W2CK,0QPRDC97rIQB3Jh3hrVJoH,1rrqJ9QkOBYJlsZgqqwxgB,1KQ0AjjjByOHxGSCfrMJ7Q,0QBzMgT7NIeoCYy3sJCof1,2i2gDpKKWjvnRTOZRhaPh2,5ww2BF9slyYgNOk37BlC4u,51TYqTZ1eTZsHVcrt0mmqe,1xK59OXxi2TAAAbmZK0kBL,1PckUlxKqWQs3RlWXVBLw3,2L9N0zZnd37dwF0clgxMGI,6Yqw4HtJuFXleJtgzYXWzT,3ydF3JegxOkpoVy9gIfpXC,4AuRsGcSsNqslLywUCWhAG,57zEYntUat0ofbFpoicN26,5CWQ9WBbd1HESogIIuTlVI,2C72bOiY3kbCgDBBznoejk,0JLLcOdBUKfDtzY0seXQHC,7KkaBto4aewGt8wl6q6wxH,4cEhCuw7sLZsLIQA1EBvxZ,1E5zyR7GT3PKjH3cqhVXVf,3ppaMjhBDxRi2kTpawOSJn,2ChLs6LH0Kc6h2f7WI2l0w,6xSuenHgNaeQDRiHr8qKpm,0iIvZ25h16B9DhjptYp8hK,2DDOQBKGmkv7bPoYF1bELz,2Ej4vdkwwBDyE6QfFWaXUF,7ILXon2PBU1bGYemfNg91q,1E5m5P8d7C5zMTkyP0BlRa,1jfpL95gwUMTPd9R3wdvaF,2VbhYoAVNbADZEPmbRhw5E,6jUscicoyUljrPOdQCfhnd,5pOY1Vg2TIIbVBfR6yAg0C,2nUVecgsUy7qfccJyn1VNh,6SHMdpkJr50

   ⚠️ Could not fetch batch 11. Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=39JofJHEtg8I4fSyo7Imft,6JVOeKaRMwFjgiH3IbZhW1,0JXXNGljqupsJaZsgSbMZV,1SRw5p2lVAi7RGlHEmZg66,6dgUya35uo964z7GZXM07g,1blLiE48uAhJAebV9dnODK,1qEmFfgcLObUfQm0j1W2CK,0QPRDC97rIQB3Jh3hrVJoH,1rrqJ9QkOBYJlsZgqqwxgB,1KQ0AjjjByOHxGSCfrMJ7Q,0QBzMgT7NIeoCYy3sJCof1,2i2gDpKKWjvnRTOZRhaPh2,5ww2BF9slyYgNOk37BlC4u,51TYqTZ1eTZsHVcrt0mmqe,1xK59OXxi2TAAAbmZK0kBL,1PckUlxKqWQs3RlWXVBLw3,2L9N0zZnd37dwF0clgxMGI,6Yqw4HtJuFXleJtgzYXWzT,3ydF3JegxOkpoVy9gIfpXC,4AuRsGcSsNqslLywUCWhAG,57zEYntUat0ofbFpoicN26,5CWQ9WBbd1HESogIIuTlVI,2C72bOiY3kbCgDBBznoejk,0JLLcOdBUKfDtzY0seXQHC,7KkaBto4aewGt8wl6q6wxH,4cEhCuw7sLZsLIQA1EBvxZ,1E5zyR7GT3PKjH3cqhVXVf,3ppaMjhBDxRi2kTpawOSJn,2ChLs6LH0Kc6h2f7WI2l0w,6xSuenHgNaeQDRiHr8qKpm,0iIvZ25h16B9DhjptYp8hK,2DDOQBKGmkv7bPoYF1bELz,2Ej4vdkwwBDyE6QfFWaXUF,7ILXon2PBU1bGYemfNg91q,1E5m5P8d7C5zMTkyP0BlRa,1jfpL95gwUMTPd9R3wdvaF,2VbhYoAVNbADZEPmbRhw5E,6jUscicoyUljrPOdQCfhnd,5pOY1Vg2TII